In [1]:
with open('tmp.txt', 'r') as f:
    lines = f.read().splitlines()

In [2]:
lines

['2.0G -rw-rw-r-- 1 jmh jmh 2.0G Jun  5 17:57 009_00005.parquet',
 '1.9G -rw-rw-r-- 1 jmh jmh 1.9G Jun  5 18:13 012_00003.parquet',
 '1.3G -rw-rw-r-- 1 jmh jmh 1.3G Jun  5 14:42 010_00006.parquet',
 '1.3G -rw-rw-r-- 1 jmh jmh 1.3G Jun  5 20:18 007_00007.parquet',
 '894M -rw-rw-r-- 1 jmh jmh 894M Jun  5 14:21 000_00001.parquet',
 '848M -rw-rw-r-- 1 jmh jmh 848M Jun  5 15:25 003_00005.parquet',
 '723M -rw-rw-r-- 1 jmh jmh 723M Jun  5 16:08 013_00006.parquet',
 '709M -rw-rw-r-- 1 jmh jmh 709M Jun  5 16:02 013_00001.parquet',
 '702M -rw-rw-r-- 1 jmh jmh 702M Jun  5 15:46 009_00002.parquet',
 '697M -rw-rw-r-- 1 jmh jmh 697M Jun  5 16:13 013_00000.parquet',
 '687M -rw-rw-r-- 1 jmh jmh 687M Jun  5 16:02 013_00002.parquet',
 '679M -rw-rw-r-- 1 jmh jmh 679M Jun  5 16:11 013_00009.parquet',
 '673M -rw-rw-r-- 1 jmh jmh 673M Jun  5 16:05 013_00004.parquet',
 '670M -rw-rw-r-- 1 jmh jmh 670M Jun  5 16:05 013_00003.parquet',
 '661M -rw-rw-r-- 1 jmh jmh 661M Jun  5 16:08 013_00005.parquet',
 '643M -rw

In [3]:
lines = [line.split(' ')[-1] for line in lines]
lines

['009_00005.parquet',
 '012_00003.parquet',
 '010_00006.parquet',
 '007_00007.parquet',
 '000_00001.parquet',
 '003_00005.parquet',
 '013_00006.parquet',
 '013_00001.parquet',
 '009_00002.parquet',
 '013_00000.parquet',
 '013_00002.parquet',
 '013_00009.parquet',
 '013_00004.parquet',
 '013_00003.parquet',
 '013_00005.parquet',
 '013_00008.parquet',
 '000_00000.parquet',
 '009_00007.parquet',
 '011_00000.parquet',
 '003_00009.parquet',
 '003_00004.parquet',
 '008_00009.parquet',
 '013_00007.parquet',
 '000_00008.parquet',
 '011_00002.parquet',
 '001_00008.parquet',
 '010_00002.parquet',
 '010_00007.parquet',
 '011_00003.parquet',
 '007_00006.parquet',
 '011_00001.parquet',
 '004_00000.parquet']

In [4]:
len(lines)

32

In [5]:
base_url = "https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/"
lines = [base_url + line for line in lines]
lines[:5]

['https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/009_00005.parquet',
 'https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/012_00003.parquet',
 'https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/010_00006.parquet',
 'https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/007_00007.parquet',
 'https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/000_00001.parquet']

In [6]:
with open("datasets/100BT.txt", "w") as f:
    f.write('\n'.join(lines))

In [7]:
with open("datasets/100BT.txt", "r") as f:
    targets = f.read().splitlines()
len(targets)

32

In [9]:
with open("downloaded_urls.txt", 'r') as f:
    downloaded_urls = f.read().splitlines()
len(downloaded_urls)

27

In [10]:
for url in targets:
    if url not in downloaded_urls:
        print(url)

https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/010_00006.parquet
https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/000_00001.parquet
https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/013_00006.parquet
https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/013_00001.parquet
https://hf-mirror.com/datasets/HuggingFaceFW/fineweb-edu/resolve/main/sample/100BT/011_00003.parquet
